# Create data

In [3]:
import re
import os

## Reading path of the data and user name
Rawpath = "..\\lib"
whatsappName = input("Enter name of the user: ")

## Reading data
data = ""
inputPath = os.path.join(Rawpath)
for file in os.listdir(inputPath):
    filePath = os.path.join(inputPath, file)
    with open(filePath, encoding = "utf8") as f:
        temp = f.read()
    data += " " + temp

## Creaitng output folder
outputPath = os.path.join(Rawpath, "Output\\")
if not os.path.exists(os.path.dirname(outputPath)):
    try:
        os.makedirs(os.path.dirname(outputPath))
    except OSError as exc: # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

## Splitting data as new message as #brk#
data = re.sub("\n[0-9]*/[0-9]*/[0-9]*, [0-9]*:[0-9]* [ap][m] - ", " #brk# ", data)
data = data.split(" #brk# ")[1:]

## Creating in and out messages
dataList = []
for line in data:
    line = line.replace("\n", " ")
    dataList.append(line.split(": "))
    
flag_0 = True
flag_1 = True
incoming = []
outgoing = []
temp = ""
for item in dataList:
    try:
        name = item[0]
        txt = item[1]
    except:
        continue
    if name == whatsappName:
        if flag_0:
            if not flag_1:
                incoming.append(temp)
            temp = txt
            flag_1 = True
            flag_0 = False
        else:
            temp += " " + txt
    else:
        if flag_1:
            if not flag_0:
                outgoing.append(temp)
            temp = txt
            flag_0 = True
            flag_1 = False
        else:
            temp += " " + txt
            

## Writting created dataset
with open(outputPath + '\\incoming.txt', 'w', encoding = "utf8") as f:
    for item in incoming:
        f.write("%s\n" % item)
with open(outputPath + '\\outgoing.txt', 'w', encoding = "utf8") as f:
    for item in outgoing:
        f.write("%s\n" % item)

Enter name of the user: Aritra Banerjee


# Preprocessing

In [8]:
import re
import os
import random
import string
import warnings
import pickle
warnings.filterwarnings("ignore")

In [6]:
# Function to remove digits, punctuation and extra space
def processed(text):
    # Removs digits
    def removeDigits(text): 
        result = re.sub(r'\d+', '', text) 
        return result 
    # Removes punctuation 
    def removePunctuation(text): 
        translator = str.maketrans('', '', string.punctuation.replace(":", "").replace("'","")) 
        return text.translate(translator) 
    # Removes whitespace from text 
    def removeSpaces(text): 
        return  " ".join(text.split())
    # Main function
    if __name__=="__main__":
        text = text.replace("<Media omitted>", " ")
        text = text.lower()
        #text = removeDigits(text)
        text = removePunctuation(text)
        text = removeSpaces(text)
        return text
def performprocess(ListOfMsg):
    output = []
    for text in ListOfMsg:
        output.append(processed(text))
    return(output)

def getWordCount(corpus):
    WordCount = {}
    for text in corpus:
        for word in text.split():
            try:
                WordCount[word] += 1
            except:
                WordCount[word] = 1 
    return WordCount

def getIndexDicMap(incoming, outgoing):
    inDic = {}
    outDic = {}
    for i in range(len(incoming)):
        if incoming[i] not in inDic.keys():
            inDic[incoming[i]] = i
    for i in range(len(outgoing)):
        if outgoing[i] not in outDic.keys():
            outDic[outgoing[i]] = i
    mappingDic = {}
    for index in range(len(incoming)):
        try:
            mappingDic[inDic[incoming[index]]].append(outDic[outgoing[index]])
        except:
            mappingDic[inDic[incoming[index]]] = [outDic[outgoing[index]]]
    return inDic, outDic, mappingDic

In [124]:
inputPath = "..\\lib\\Output\\"
outputPath = "..\\lib\\Output\\vocab\\"

with open(inputPath + "incoming.txt", encoding = "utf8") as f:
    incoming = f.read()
incoming = incoming.split("\n")        
with open(inputPath + "outgoing.txt", encoding = "utf8") as f:
    outgoing = f.read()
outgoing = outgoing.split("\n")

inProcessed = performprocess(incoming)
outProcessed = performprocess(outgoing)

inWordCount = getWordCount(inProcessed)

inDic, outDic, mappingDic = getIndexDicMap(inProcessed, outProcessed)
outDicMap = dict([(value, key) for key, value in outDic.items()]) 

In [126]:
if not os.path.exists(os.path.dirname(outputPath)):
    try:
        os.makedirs(os.path.dirname(outputPath))
    except OSError as exc: # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise            
            
with open(outputPath + "inProcessed.p", 'wb') as f:
    pickle.dump(inProcessed, f)
with open(outputPath + "outProcessed.p", 'wb') as f:
    pickle.dump(outProcessed, f)
with open(outputPath + "inWordCount.p", 'wb') as f:
    pickle.dump(inWordCount, f)
with open(outputPath + "inDic.p", 'wb') as f:
    pickle.dump(inDic, f)
with open(outputPath + "mappingDic.p", 'wb') as f:
    pickle.dump(mappingDic, f)
with open(outputPath + "outDicMap.p", 'wb') as f:
    pickle.dump(outDicMap, f)

# Model

In [1]:
import re
import os
import scipy
import random
import pickle 
import warnings
import BuildVocab
import numpy as np
warnings.filterwarnings("ignore")
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

In [18]:
class Model():
    def __init__(self):
        self.SIZE = 100
        self.WINDOW = 5
        self.ALPHA = 0.001
        self.Method = 1

        self.inputPath = "..\\lib\\Output\\vocab\\"
        self.outputPath = "..\\lib\\Output\\weights\\"

        with open(self.inputPath + "inProcessed.p", 'rb') as f:
            self.inProcessed = pickle.load(f)
#         with open(self.inputPath + "outProcessed.p", 'rb') as f:
#             self.outProcessed = pickle.load(f)
        with open(self.inputPath + "inWordCount.p", 'rb') as f:
            self.inWordCount = pickle.load(f)
        with open(self.inputPath + "inDic.p", 'rb') as f:
            self.inDic = pickle.load(f)
        with open(self.inputPath + "mappingDic.p", 'rb') as f:
            self.mappingDic = pickle.load(f)
        with open(self.inputPath + "outDicMap.p", 'rb') as f:
            self.outDicMap = pickle.load(f)

        if not os.path.exists(os.path.dirname(self.outputPath)): 
            try:
                os.makedirs(os.path.dirname(self.outputPath))
            except OSError as exc: # Guard against race condition
                if exc.errno != errno.EEXIST:
                    raise
            print("Model training under process...")
            self.model = self.trainModel(self.inProcessed)
            self.model.wv.save_word2vec_format(self.outputPath + "word2vecWeights.txt", binary=False)  
        else:
            print("Trained weights found\nLoading..")
            self.model = KeyedVectors.load_word2vec_format(self.outputPath + "word2vecWeights.txt", binary=False)
    ## Function to training a word2vec model    
    def trainModel(self):
        trainData = list(i.split() for i in self.inProcessed)
        model = Word2Vec(trainData, size = self.Size, window = self.Window, min_count = 1, workers = 4)
        return model  
    
    ## Function to return the word embedding of the word, zeros if the wor is unknown
    def Vec(self, word):
        try:
            return self.model[word]
        except:
            return np.zeros(self.SIZE)
        
    ## Function to return the count of the word in the corpus   
    def Count(self, word):
        try:
            return self.inWordCount[word]
        except:
            return 0
        
    ## Function to return the similarity distance (cosine) between two senetnce.
    ## 0 implies they are totally similar and 1 implies they are way apart
    def CosSifDist(self, sentence_1, sentence_2):
        vector_1 = np.sum([self.Vec(word)*(self.ALPHA / (self.ALPHA + self.Count(word))) for word in BuildVocab.processed(sentence_1).split()], axis = 0)
        vector_2 = np.sum([self.Vec(word)*(self.ALPHA / (self.ALPHA + self.Count(word))) for word in BuildVocab.processed(sentence_2).split()], axis = 0)
        cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
        #print('Word Embedding method with a cosine distance method, sentences are similar to',round((1-cosine)*100,2),'%')
        return cosine  
    
    ## Function tto return the similarity distance (word mover) between two senetnce.
    def WMDist(sentence_1, sentence_2):
        sentence_1 = BuildVocab.processed(sentence_1)
        sentence_2 = BuildVocab.processed(sentence_2)
        return self.model.wmdistance(sentence_1, sentence_2)    
    
    ## Function to return the closest messages given a raw text
    def getCloseMsg(self, RawText):
        if self.Method == 1:
            temp = np.inf
            closestMsg = None
            for oldmsg in self.inProcessed:
                similarity = self.CosSifDist(RawText, oldmsg)
                if similarity < temp:
                    temp = similarity
                    closestMsg = oldmsg
        elif self.Method == 2:
            temp = np.inf
            closestMsg = None
            for oldmsg in self.inProcessed:
                similarity = self.WMDist(RawText, oldmsg)
                if similarity < temp:
                    temp = similarity
                    closestMsg = oldmsg
        return closestMsg 
    
    ## Function to return the reply of a message
    def getReply(self, newMsg):
        replies = []
        msg = self.getCloseMsg(newMsg)
        index = self.inDic[msg]
        repliesindex = self.mappingDic[index]
        for i in repliesindex:
            replies.append(self.outDicMap[i])
        return(random.choice(replies))

# Main

In [16]:
%autoreload
import utils.bot as bot

model = bot.Model()

Vocabulary found
Loading...
Trained weights found
Loading..
Weights loaded successfully


In [17]:
model.getReply("Hola")

'🤖 -> Sorry, I didnot get you..'